In [132]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [133]:
from tensorflow.keras import Layer
from tensorflow import keras
from keras.api.models import Sequential

In [134]:
class CustomDenseLayer(Layer):
    def __init__(self, units=64):
        super(CustomDenseLayer, self).__init__()
        self.units = units
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer = 'random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer = 'zeros',
                                 trainable=True)
    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b)

In [135]:
from tensorflow.keras.layers import Softmax

model = Sequential([
    CustomDenseLayer(128),
    CustomDenseLayer(32),
    CustomDenseLayer(8),
])

In [136]:
model.compile(
    optimizer='sgd',
    loss='mse',
    metrics=[keras.metrics.RootMeanSquaredError()])

In [137]:
import pandas as pd
import numpy as np

df = pd.read_csv('./londonhousing_regression.csv')
print(df.shape)

(1071, 12)


In [138]:
print(df.columns)

Index(['code', 'area', 'date', 'median_salary', 'life_satisfaction',
       'mean_salary', 'recycling_pct', 'population_size', 'number_of_jobs',
       'area_size', 'no_of_houses', 'borough_flag'],
      dtype='object')


In [139]:
from sklearn.preprocessing import LabelEncoder

df['mean_salary'] = df['mean_salary'].replace('#', np.nan)
df['mean_salary'] = df['mean_salary'].replace('-', np.nan)
df.dropna(inplace=True)

le = LabelEncoder()
x = df.iloc[:, :-1]
y = df.iloc[:, -1]

x.drop(columns='code', inplace=True)
x['year'] = x['date'].str.split('-', expand=True)[0]
x.drop(columns='date', inplace=True)
x['area'] = le.fit_transform(x['area'])

In [140]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [141]:
model.build((None, x_train.shape[1]))

In [142]:
model.fit(x_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - loss: 153208.7656 - root_mean_squared_error: 351.8931
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 658us/step - loss: 0.9459 - root_mean_squared_error: 0.9726
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 0.9526 - root_mean_squared_error: 0.9760
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 640us/step - loss: 0.9522 - root_mean_squared_error: 0.9758
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - loss: 0.9460 - root_mean_squared_error: 0.9726
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 641us/step - loss: 0.9180 - root_mean_squared_error: 0.9580
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - loss: 0.9505 - root_mean_squared_error: 0.9749
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - loss: 0.9349 - root_mean_squared_error: 0.9669
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step - loss: 0.9462 - root_mean_squared_error: 0.9727
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - loss: 0.9396 - root_mean_squa

In [143]:
mse = model.evaluate(x_test, y_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9197 - root_mean_squared_error: 0.9590


In [144]:
print(mse)

[0.9253731369972229, 0.9619631767272949]
